# Notification

> This notebook implement notification workflow

In [ ]:
#| default_exp notification

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from threading import Thread, Event
import time
from typing import Optional, Callable
from llmcam.fn_to_fc import complete, tool_schema

In [ ]:
#| export
# Define the stream thread class
class StreamThread(Thread):
    def __init__(self, thread_id, tools, messages):
        super().__init__()
        self.thread_id = thread_id
        self.stop_event = Event()
        self.tools = tools
        self.messages = messages

    def run(self):
        while not self.stop_event.is_set():
            complete(self.messages, tools=self.tools)
            time.sleep(5)

    def stop(self):
        self.stop_event.set()

In [ ]:
#| export
def notification_stream_core(
    tools: list,  # Tools to use
    send_notification: Callable,  # Function to send the notification
    messages: list,  # Previous conversation with the user
    stream_starter: Optional[Callable] = None,  # Function to start the stream
    stream_stopper: Optional[Callable] = None  # Function to stop the stream
) -> str:
    """Core function to start and stop the notifications stream"""
    # Copy the messages to avoid modifying the original list
    submessages = [ message for message in messages ]

    # Add sending notification services to tool schema
    subtools = [ tool for tool in tools if tool['function']['name'] != 'start_notification_stream' ]
    subtools.append(tool_schema(send_notification, 'send_notification'))
    subtools.append(tool_schema(stream_stopper, 'send_notification'))

    # Start the notifications stream
    stream_starter(subtools, submessages)

    return 'Notifications stream started'

In [ ]:
#| export
def default_stream_starter(tools, messages):
    """Default function to start the notifications stream"""
    global stream_thread

    # Start the notifications stream
    stream_thread = StreamThread(1, tools, messages)
    stream_thread.start()

def default_stream_stopper():
    """Default function to stop the notifications stream"""
    global stream_thread

    # Stop the notifications stream
    stream_thread.stop()
    stream_thread.join()

In [ ]:
#| export
def process_notification_schema(
    start_notifications_stream: Callable,  # Function to start the notifications stream
):
    """Process the notification schema"""
    notification_schema = tool_schema(start_notifications_stream, 'notification')

    notification_schema['function']['parameters'] = {
        'type': 'object',
        'properties': {
            'messages': {
                'description': 'All the previous messages in the conversation',
                'type': 'array',
                'items': {
                    'type': 'object',
                    'properties': {
                        'role': {
                            'type': 'string',
                            'enum': ['user', 'tool', 'system', 'assistant']
                        },
                        'content': {
                            'type': 'string'
                        }
                    }
                }
            }
        }
    }
    return notification_schema

Simulated workflow:

In [ ]:
from llmcam.fn_to_fc import complete, form_msgs, form_msg, tool_schema, print_msgs
import random

In [ ]:
tools = []

In [ ]:
def random_generator():
    """Generate a random number between 1 and 100"""
    return random.randint(1, 100)

In [ ]:
notifications = []
def send_notification(msg: str):
    """Send a notification"""
    notifications.append(msg)
    return notifications

In [ ]:
def start_notification_stream(
    messages: list  # Previous conversation with the user
):
    return notification_stream_core(
        tools, 
        send_notification, 
        messages, 
        default_stream_starter, 
        default_stream_stopper
    )

In [ ]:
tools.append(tool_schema(random_generator, 'random_generator'))
tools.append(process_notification_schema(start_notification_stream))

In [ ]:
tools

[{'type': 'function',
  'function': {'name': 'random_generator',
   'description': 'Generate a random number between 1 and 100',
   'parameters': {'type': 'object', 'properties': {}, 'required': []},
   'metadata': {'module': '__main__', 'service': 'random_generator'}}},
 {'type': 'function',
  'function': {'name': 'start_notification_stream',
   'description': 'No description provided.',
   'parameters': {'type': 'object',
    'properties': {'messages': {'description': 'All the previous messages in the conversation',
      'type': 'array',
      'items': {'type': 'object',
       'properties': {'role': {'type': 'string',
         'enum': ['user', 'tool', 'system', 'assistant']},
        'content': {'type': 'string'}}}}}},
   'metadata': {'module': '__main__', 'service': 'notification'}}}]

In [ ]:
#| eval: false
messages = form_msgs([
    ('system', 'You are a helpful system administrator. Use the supplied tools to assist the user.'),
    ('user', 'Can you notify me every time you generate a number higher than 50? Stop after 10 notifications.'),
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to assist the user.
>> User:
Can you notify me every time you generate a number higher than 50? Stop after 10 notifications.
>> Assistant:
The notification system has been set up. Here are the generated numbers higher than 50:  1. 85 2. 87
3. 69 4. 67 5. 84  Notifications will stop after reaching 10 notifications. If you'd like to
continue, please let me know!


In [ ]:
#| eval: false
messages

[{'role': 'system',
  'content': 'You are a helpful system administrator. Use the supplied tools to assist the user.'},
 {'role': 'user',
  'content': 'Can you notify me every time you generate a number higher than 50? Stop after 10 notifications.'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_UZUZOGh7lTqDiGlNHe8ZHOY6',
    'function': {'arguments': '{}', 'name': 'random_generator'},
    'type': 'function'},
   {'id': 'call_mbMa20rYi5YELniX5I5pBxAi',
    'function': {'arguments': '{"messages": [{"role": "user", "content": "Can you notify me every time you generate a number higher than 50? Stop after 10 notifications."}]}',
     'name': 'start_notification_stream'},
    'type': 'function'},
   {'id': 'call_24apEIMuzDaKRBibE8O8xb7o',
    'function': {'arguments': '{}', 'name': 'random_generator'},
    'type': 'function'},
   {'id': 'call_o7RW5pg4LnOZtHq2tBsQ1WTA',
    'function': {'arguments': '{}', 'name': 'random_generator'},
    'type': '

In [ ]:
#| eval: false
for noti in notifications:
    print(noti)

Generated number: 59, which is higher than 50.
Generated number: 81, which is higher than 50.
Generated number: 58, which is higher than 50.
Generated number: 90, which is higher than 50.
Generated number: 86, which is higher than 50.
Generated number: 88, which is higher than 50.
Generated number: 95, which is higher than 50.
Generated number: 90, which is higher than 50.
Generated number: 91, which is higher than 50.
Generated number: 97, which is higher than 50.


In [ ]:
#| eval: false
len(notifications)

10

In [ ]:
#| eval: false
stream_thread.is_alive()

False

In [ ]:
#| eval: false
stream_thread.tools

[{'type': 'function',
  'function': {'name': 'random_generator',
   'description': 'Generate a random number between 1 and 100',
   'parameters': {'type': 'object', 'properties': {}, 'required': []},
   'metadata': {'module': '__main__', 'service': 'random_generator'}}},
 {'type': 'function',
  'function': {'name': 'send_notification',
   'description': 'Send a notification',
   'parameters': {'type': 'object',
    'properties': {'msg': {'type': 'string',
      'description': 'No description provided.'}},
    'required': ['msg']},
   'metadata': {'module': '__main__', 'service': 'send_notification'}}},
 {'type': 'function',
  'function': {'name': 'default_stream_stopper',
   'description': 'Default function to stop the notifications stream',
   'parameters': {'type': 'object', 'properties': {}, 'required': []},
   'metadata': {'module': '__main__', 'service': 'send_notification'}}}]

In [ ]:
#| eval: false
stream_thread.messages

[{'role': 'user',
  'content': 'Can you notify me every time you generate a number higher than 50? Stop after 10 notifications.'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_X0ssVOJQnhFwQDXdplJx0x97',
    'function': {'arguments': '{}', 'name': 'random_generator'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"random_generator": 59}',
  'tool_call_id': 'call_X0ssVOJQnhFwQDXdplJx0x97'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_3bJTvIzUoWO22fK7JDhi2LGA',
    'function': {'arguments': '{"msg":"Generated number: 59, which is higher than 50."}',
     'name': 'send_notification'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"msg": "Generated number: 59, which is higher than 50.", "send_notification": ["Generated number: 59, which is higher than 50."]}',
  'tool_call_id': 'call_3bJTvIzUoWO22fK7JDhi2LGA'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()